In [1]:
import numpy as np
from csv import writer
import json

In [64]:
data = {}
with open('./results/2022-12-16 163709.548510.json') as result_file:
    data = json.load(result_file)

In [63]:
print(len(data['results']))

8


In [53]:
"""
data {
    'distance_matrix_time': distance_matrix_time,
    'employees': coordinates_list,
    'results': [
         'num_vehicles': n_shuttle,
         'max_time_shuttle': max_time,
         'result': {
            'routes': [{
                'route': [],
                'routeTotalTime': 0,
                'routeTotalDist': 0
                },
                ... more routes
            ]
        'objectiveTime': 0,
        'objectiveDist': 0
            },
        ... more results
        ]
    ]
}
""";

In [ ]:
dist_matrix = [[0, 1817, 1420, 1473, 1039, 882, 1608, 1431, 1507, 904],
                [1883, 0, 2113, 970, 1858, 1179, 537, 2046, 1010, 1737],
                [1457, 2022, 0, 1322, 457, 1399, 2039, 85, 1431, 2127],
                [1571, 813, 1350, 0, 1189, 858, 879, 1285, 199, 1432],
                [1074, 1867, 473, 1191, 0, 1004, 1649, 481, 1287, 1752],
                [1005, 1193, 1423, 856, 1042, 0, 1010, 1430, 896, 799],
                [1702, 494, 2056, 914, 1714, 982, 0, 1973, 952, 1468],
                [1459, 1952, 102, 1257, 454, 1388, 1973, 0, 1355, 2132],
                [1643, 881, 1414, 215, 1256, 933, 943, 1339, 0, 1511],
                [902, 1728, 2060, 1386, 1689, 761, 1382, 2069, 1422, 0]] #[m]

time_matrix = [[0, 1817, 1420, 1473, 1039, 882, 1608, 1431, 1507, 904],
                [1883, 0, 2113, 970, 1858, 1179, 537, 2046, 1010, 1737],
                [1457, 2022, 0, 1322, 457, 1399, 2039, 85, 1431, 2127],
                [1571, 813, 1350, 0, 1189, 858, 879, 1285, 199, 1432],
                [1074, 1867, 473, 1191, 0, 1004, 1649, 481, 1287, 1752],
                [1005, 1193, 1423, 856, 1042, 0, 1010, 1430, 896, 799],
                [1702, 494, 2056, 914, 1714, 982, 0, 1973, 952, 1468],
                [1459, 1952, 102, 1257, 454, 1388, 1973, 0, 1355, 2132],
                [1643, 881, 1414, 215, 1256, 933, 943, 1339, 0, 1511],
                [902, 1728, 2060, 1386, 1689, 761, 1382, 2069, 1422, 0]] #[s]

dist_matrix = np.array(dist_matrix)
time_matrix = np.array(time_matrix)


In [ ]:
vrp_out = {'routes': [
    {'route': [0, 4, 5, 8, 0],
     'routeTotalTime': 5369,
     'routeTotalDist': 3985
    }, 
    {'route': [0, 1, 6, 2, 0],
     'routeTotalTime': 8133,
     'routeTotalDist': 1806
    }, 
    {'route': [0, 3, 7, 0],
     'routeTotalTime': 8042,
     'routeTotalDist': 3564
    }, 
    {'route': [0, 9, 0],
     'routeTotalTime': 5316,
     'routeTotalDist': 3033
    }], 
 'objective': 366036
}


In [65]:
### Dist & Emissions avoided 
# Inputs :  dist_matrix: the distance matrix
#           vrp_out: the results of the vrp
# Outputs:  - the avoided distance in meters
#           - the avoided emissions in gCO2eq

def fct_avoided_dist(dist_matrix, vrp_out):
    EMISSION_CAR     = 0.17 #[g/(m*passanger)]
    EMISSION_SHUTTLE = 0.10 #[g/(m*passanger)]

    FIRST_EMPLOYEE = 1
    COMPANY_COLUMN = 0

    avoided_dist = 0
    em_shuttles  = 0
    em_cars      = 0
    nb_employees = len(dist_matrix)

    for s in range(len(vrp_out['routes'])):
        
        dist_cars = 0
        for i in range(FIRST_EMPLOYEE, len(vrp_out['routes'][s]['route'])-1):
            dist_cars += dist_matrix[i][COMPANY_COLUMN]
        
        avoided_dist += (dist_cars - vrp_out['routes'][s]['routeTotalDist'])
        em_shuttles  += (vrp_out['routes'][s]['routeTotalDist'] * EMISSION_SHUTTLE)
        em_cars      += (dist_cars * EMISSION_CAR)

    avoided_em = (em_cars - em_shuttles)*nb_employees

    return(avoided_dist, avoided_em)



In [66]:
### Overall time of work loss
# Inputs :  time_matrix: the time matrix between all employees
#           vrp_time: the time result of the vrp
# Outputs:  - the overall time lost in seconds
#           - mean time lost
#           - the maximum time lost for all employees
#           - the minimum time lost for all employees

def fct_time_lost(time_matrix, vrp_out):
    FIRST_SHUTTLE  = 0
    FIRST_EMPLOYEE = 1
    COMPANY_COLUMN = 0
    
    tot_time_lost = 0
    max_time_lost = 0
    min_time_lost = vrp_out['routes'][FIRST_SHUTTLE]['routeTotalTime'] - time_matrix[vrp_out['routes'][FIRST_SHUTTLE]['route'][FIRST_EMPLOYEE]][COMPANY_COLUMN]
    nb_employees  = len(time_matrix)

    for s in range(len(vrp_out['routes'])):
        for i in range(FIRST_EMPLOYEE, len(vrp_out['routes'][s]['route'])-1):
            time_lost_i = (vrp_out['routes'][s]['routeTotalTime'] - time_matrix[i][COMPANY_COLUMN])
            tot_time_lost  += time_lost_i

            if time_lost_i > max_time_lost:
                max_time_lost = time_lost_i
            
            if time_lost_i < min_time_lost:
                min_time_lost = time_lost_i

    mean_time_lost = tot_time_lost/nb_employees

    return(tot_time_lost, mean_time_lost, max_time_lost, min_time_lost)

In [68]:
### Calculate & save resutls
# Inputs:   create_file: a boolean : true to create a new file (and erase previous one), False to add a row to the existing file
#           test_number: the name of the test, it will apprear on the first column of the output
#           dist_matrix: the distance matrix
#           time_matrix: the time matrix
#           vrp_out: the output dictionnary of the vrp
# Output:   - a print of the costs
#           - a file called "outputs-csv" with the costs
def fct_print_costs_save(create_file, test_number, dist_matrix, time_matrix, vrp_out):
    if create_file == True:
        with open('outputsOFF2.csv', 'w', newline='') as file:
            # 2. step
            w = writer(file)
            # 3. step
            w.writerow(["Test number; total dist[m]; total time[s]; avoided dist [m]; avoided emissions [gCO2eq]; total time lost [s]; mean time lost [s]; max_time_lost [s]; min_time_lost[s]"])


    [avoided_dist, avoided_em] = fct_avoided_dist(dist_matrix, vrp_out)
    [tot_time_lost, mean_time_lost, max_time_lost, min_time_lost] = fct_time_lost(time_matrix, vrp_out)
    newrow = [str(test_number)+ ";" + str(vrp_out["objectiveDist"]) + ";" + str(vrp_out["objectiveTime"]) + ";" + str(avoided_dist)+";"+str(avoided_em)+";"+str(tot_time_lost)+";"+str(mean_time_lost)+";"+str(max_time_lost)+";"+str(min_time_lost)]

    with open(f'./results/metrics-outputsOFF2.csv', 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(newrow)

    print(
    "Total distance ---------- " + str(vrp_out["objectiveDist"]) + " [m]\n"
    "Total time -------------- " + str(vrp_out["objectiveTime"]) + " [s]\n"
    "avoided distance -------- " + str(avoided_dist) + " [m]\n"
    "avoided emissions ------- " + str(avoided_dist) + " [gCO2eq]\n"
    "total time lost --------- " + str(tot_time_lost) + " [s]\n"
    "mean time lost ---------- " + str(mean_time_lost) + " [s]\n"
    "maximum time lost ------- " + str(max_time_lost) + " [s]\n"
    "minimum time lost ------- " + str(min_time_lost) + " [s]\n"
    )

In [69]:
print(' '.join(["Test 1:", str(data['results'][0]['num_vehicles']), "shuttles,", str(data['results'][0]['max_time_shuttle']), "max."]))

Test 1: 4 shuttles, 6000 max.


In [70]:
def export_result(dfjson):
    title=' '.join(["Test 1:", str(dfjson['results'][0]['num_vehicles']), "shuttles,", str(dfjson['results'][0]['max_time_shuttle']), "max."])

    fct_print_costs_save(True, title,dfjson['distance_matrix_distance'],dfjson['distance_matrix_time'],dfjson['results'][0]['result'])

    for i in range(1,len(data['results'])):
        title=' '.join(["Test:",str(i+1), str(dfjson['results'][i]['num_vehicles']), "shuttles,", str(dfjson['results'][i]['max_time_shuttle']), "max."])
        fct_print_costs_save(True, title ,dfjson['distance_matrix_distance'],dfjson['distance_matrix_time'],dfjson['results'][i]['result'])


In [71]:
export_result(data)

Total distance ---------- 283699 [m]
Total time -------------- 21534 [s]
avoided distance -------- 188107 [m]
avoided emissions ------- 188107 [gCO2eq]
total time lost --------- 93337 [s]
mean time lost ---------- 3733.48 [s]
maximum time lost ------- 4817 [s]
minimum time lost ------- 3229 [s]

Total distance ---------- 284445 [m]
Total time -------------- 21324 [s]
avoided distance -------- 187361 [m]
avoided emissions ------- 187361 [gCO2eq]
total time lost --------- 91169 [s]
mean time lost ---------- 3646.76 [s]
maximum time lost ------- 4845 [s]
minimum time lost ------- 2908 [s]

Total distance ---------- 274294 [m]
Total time -------------- 21696 [s]
avoided distance -------- 212489 [m]
avoided emissions ------- 212489 [gCO2eq]
total time lost --------- 66111 [s]
mean time lost ---------- 2644.44 [s]
maximum time lost ------- 4183 [s]
minimum time lost ------- 1566 [s]

Total distance ---------- 302183 [m]
Total time -------------- 23045 [s]
avoided distance -------- 190412 [m]

## Sceanrio 3

In [124]:
print("------------------------------------------------")
print(' '.join([ str(data['results'][2]['num_vehicles']), "shuttles,", str(data['results'][2]['max_time_shuttle']), "max."]))
for j in range(len(data['results'][2]['result']['routes'])):
    print("----------------------")
    print("route:",data['results'][2]['result']['routes'][j]['route'], len(data['results'][2]['result']['routes'][j]['route'])-2)
    print("travel distance:",data['results'][2]['result']['routes'][j]['routeTotalDist'])
    print("travel time:",data['results'][2]['result']['routes'][j]['routeTotalTime'])
print("------------------------------------------------")

------------------------------------------------
5 shuttles, 6000 max.
----------------------
route: [0, 9, 21, 4, 2, 1, 0] 5
travel distance: 55919
travel time: 4621
----------------------
route: [0, 11, 17, 12, 16, 22, 0] 5
travel distance: 49224
travel time: 3795
----------------------
route: [0, 20, 14, 8, 3, 19, 0] 5
travel distance: 61399
travel time: 4770
----------------------
route: [0, 23, 15, 18, 6, 0] 4
travel distance: 61157
travel time: 4816
----------------------
route: [0, 13, 5, 24, 7, 10, 0] 5
travel distance: 46595
travel time: 3694
------------------------------------------------


## Scenario 7

In [121]:
print("------------------------------------------------")
print(' '.join([ str(data['results'][6]['num_vehicles']), "shuttles,", str(data['results'][6]['max_time_shuttle']), "max."]))
for j in range(len(data['results'][6]['result']['routes'])):
    print("----------------------")
    print("route:",data['results'][6]['result']['routes'][j]['route'], len(data['results'][6]['result']['routes'][j]['route'])-2)
    print("travel distance:",data['results'][6]['result']['routes'][j]['routeTotalDist'])
    print("travel time:",data['results'][6]['result']['routes'][j]['routeTotalTime'])
print("------------------------------------------------")

------------------------------------------------
8 shuttles, 6000 max.
----------------------
route: [0, 11, 17, 12, 16, 22, 0] 5
travel distance: 49224
travel time: 3795
----------------------
route: [0, 9, 15, 0] 2
travel distance: 52051
travel time: 3297
----------------------
route: [0, 3, 19, 0] 2
travel distance: 53720
travel time: 4188
----------------------
route: [0, 4, 2, 1, 0] 3
travel distance: 61773
travel time: 3972
----------------------
route: [0, 23, 21, 0] 2
travel distance: 51353
travel time: 3860
----------------------
route: [0, 20, 14, 8, 0] 3
travel distance: 50704
travel time: 3721
----------------------
route: [0, 18, 6, 0] 2
travel distance: 42999
travel time: 3061
----------------------
route: [0, 13, 5, 24, 7, 10, 0] 5
travel distance: 46595
travel time: 3694
------------------------------------------------


## Scenario 5

In [123]:
print("------------------------------------------------")
print(' '.join([ str(data['results'][4]['num_vehicles']), "shuttles,", str(data['results'][4]['max_time_shuttle']), "max."]))
for j in range(len(data['results'][4]['result']['routes'])):
    print("----------------------")
    print("route:",data['results'][4]['result']['routes'][j]['route'], len(data['results'][4]['result']['routes'][j]['route'])-2)
    print("travel distance:",data['results'][4]['result']['routes'][j]['routeTotalDist'])
    print("travel time:",data['results'][4]['result']['routes'][j]['routeTotalTime'])
print("------------------------------------------------")

------------------------------------------------
6 shuttles, 6000 max.
----------------------
route: [0, 23, 20, 19, 22, 0] 4
travel distance: 46669
travel time: 3799
----------------------
route: [0, 21, 4, 2, 1, 0] 4
travel distance: 54555
travel time: 4466
----------------------
route: [0, 11, 12, 16, 17, 0] 4
travel distance: 47914
travel time: 3320
----------------------
route: [0, 14, 3, 8, 0] 3
travel distance: 60207
travel time: 4419
----------------------
route: [0, 9, 15, 18, 6, 0] 4
travel distance: 59457
travel time: 4393
----------------------
route: [0, 13, 5, 24, 7, 10, 0] 5
travel distance: 46595
travel time: 3694
------------------------------------------------
